# Sprint 8 - Web Scrapping (Ejercicios)

En el siguente ejericicio se busca utilizar **Python** como una herramienta para automatizar la extracción de datos desde una página web. Esta actividad se conoce como **WEB SCRAPPING** y es de mucha utilidad cuando no se cuenta con fuentes primarias de datos, o con datos no estructurados en internet.

En este sentido, la página que se va a consultar es una proveniente del dominio *booking.com* en la cual se muestran diferentes habitaciones disponibles en una ciudad cualquiera y para una fecha determinada.

**ADVERTENCIA:** El siguiente ejercicio está basado en la versión del dominio booking.com de Octubre - 2025. En caso que los administradores hagan cambios a nivel de su página, las salidas de las diferentes funciones y comandos podrían presentar errores o resultados no esperados. A pesar de esto, los resultados aquí alcanzados son consistentes y reproducibles con los ajustes en los argumentos de los métodos y funciones del caso.

## Entendimiento del contexto

El sector de hotelería en latinoamérica ha tenido un importante crecimiento en los últimos 50 años gracias a la implementación de políticas públicas focalizadas en incentivar el turismo y la conservación del medio ambiente. Lo anterior ha derivado en la llegada de capitales locales y extranjeros para la construcción y puesta en marcha de hoteles, lodges, hosterías y resorts en sitios de alta demanda turística.

Adicional a esto, la mayor conectividad y el desarrollo económico de la región ha incrementado aún más la necesidad de contar con sitios de hospedaje, visto que cada vez existen más viajeros de negocios que buscan de lugares cómodos, céntricos y seguros para pasar las noches durante sus estancias fuera de su lugar de residencia.

A pesar de esto, uno de los puntos pendientes de desarrollo a nivel de la industria hace referencia a la disponibilidad de información granular de mercado en lo referente a precios, ocupación, y características de habitaciones. De hecho, se podría decir que existe una suerte de recelo por compartir datos entre competidores, lo cual es comprensible considerando la etapa aún inmadura en la que se encuentra el sector, donde muchas empresas son aún administradas o pertenecen a grupos familiares o cooperativos.

Visto esto, una de estas empresas te ha contratado para que les ayudes obteniendo esta información a través de tus conocimientos en **Web Scrapping**. Lo anterior, considerando que actualmente existen muchas páginas web de operadores que ofertan vuelos, hoteles y actividades a modo de "Agencias de Viajes Online" (OTAs), y cuentan con datos transversales de varios sitios de alojamiento por ciudad y fecha.

Booking es la principal OTA a nivel mundial, y por eso vamos a aprovechar de su accesibilidad en toda la región para este proceso de extracción.

## Preparación del scrapper

En primera instancia carga las librerías que se van a utilizar para desarrollar el **web scrapper**. Existen 3 que son importantes para tu consideración:

* **requests**: Contiene funciones y métodos para interactuar con sitios web y su código subyacente (*html*).
* **re**: Contiene funciones y métodos para trabajar con expresiones regulares en textos. En cuanto a esto, es muy común tener que procesar textos desde fuentes web.
* **bs4**: Contiene la función **BeautifulSoup** que estructura el código *html* para que sea más fácil su interpretación.
* **datetime**: Contiene las funciones **date** y **timedelta** que facilitan la generación y cálculos con fechas.

Recuerda también cargar otras que has venido utilizando como **pandas** y **numpy**.

In [56]:
# Cargar librerías
import pandas as pd
import numpy as np
from datetime import date, timedelta 

import requests
import re
from bs4 import BeautifulSoup

Para que el scrapper a desarrollar funcione correctamente, es necesario preparar todos los parámetros pertinentes. En concreto, tómese como referencia esta url de ejemplo asociada a una consulta en booking.com por habitaciones de hoteles en Quito - Ecuador:

https://www.booking.com/searchresults.es.html?ss=Quito%2C+Ecuador&dest_type=city&checkin=2026-11-08&checkout=2026-11-09&group_adults=2&no_rooms=1&group_children=0&nflt=price%3DUSD%3Bclass%3D5

Observa cómo se compone esta url:

* booking.com: Esto corresponde al dominio web de la OTA.
* searchresults.es.html?: Esto es un comando que especifica que se está haciendo una búsqueda.
* ss=Quito%2C+Ecuador: Esto indica el sitio geográfico de la búsqueda.
* dest_type=city: Esto indica que el sitio geográfico buscado corresponde a una ciudad.
* checkin=2026-11-08: Esto indica la fecha de entrada buscada.
* checkout=2026-11-09: Esto indica la fecha de salida buscada.
* group_adults=2: Esto indica el número de adultos que usarían la habitación.
* no_rooms=1: Esto indica el número de habitaciones buscadas.
* group_children=0: Esto indica el número de niños que usarían la habitación.
* price%3DUSD: Esto indica que la búsqueda debe arrojar precios en USD.
* class%3D5: Esto indica que la búsqueda solo debe incluir alojamientos de 5 estrellas.

Entonces, convendría parametrizar todos los componentes que deseamos controlar. Por ahora nos concentraremos en el sitio geográfico, las fechas de entrada y salida, el número de adultos y niños, y la cantidad de habitaciones. 

Crea entonces una variable llamada `place` que contenga la ubicación geográfica de una búsqueda en el sitio web. Utiliza Quito - Ecuador como referencia

In [57]:
ciudad = "Quito"
pais = "Ecuador"
place = "%2C+".join([ciudad, pais])

Crea la variable de texto `checkin` que guarde la fecha de entrada de una búsqueda en el sitio web. Puedes utilizar una fecha cualquiera, siempre y cuando sea futura al día de hoy y utilices formato ISO (yyyy-mm-dd).

In [58]:
checkin = "2026-05-04"

Define la variable de texto `checkout` como el día siguiente de la variable anterior.

In [59]:
checkout = date.fromisoformat(checkin) + timedelta(1)
checkout = checkout.strftime("%Y-%m-%d")

Crea las variables `adults`, `children` y `rooms` que contengan el número de adultos, niños y habitaciones de una búsqueda en el sitio web.

In [60]:
adults = 2
children = 0
rooms = 1

Parametriza la url de la búsqueda considerando las variables antes descritas.

In [61]:
url_consulta = "https://www.booking.com/searchresults.es.html?ss={}&dest_type=city&checkin={}&checkout={}&group_adults={}&no_rooms={}&group_children={}&nflt=price%3DUSD%3Bclass%3D5".format(place, checkin, checkout, adults, rooms, children)
print(url_consulta)

https://www.booking.com/searchresults.es.html?ss=Quito%2C+Ecuador&dest_type=city&checkin=2026-05-04&checkout=2026-05-05&group_adults=2&no_rooms=1&group_children=0&nflt=price%3DUSD%3Bclass%3D5


Finalmente, es conveniente definir una cabecera. La misma corresponde a una serie de características que especifican el tipo de usuario o agente que está realizando la consulta. Esto es necesario, ya que muchos dominios ponen restricciones cuando quien accede a una url es un robot o un código malicioso. Replica por tanto este código:

```py
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36",
    "Accept-Language": "en-US, en;q=0.5"
}
```

In [62]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36",
    "Accept-Language": "en-US, en;q=0.5"
}

## Ejecución de consulta

Teniendo ya nuestra url parametrizada es posible hacer una consulta desde **Python** hacia el sitio web. Haslo utilizando las función `requests.get` y la plantilla de código a continuación, guarda el resultado en la variable `respuesta` e imprímela. 

```py
estado = ""
rep = 0

while estado != "OK" and rep < 20:
    respuesta = # Aquí tu código #
    estado = respuesta.reason
    rep += 1

print(respuesta)
print(respuesta.reason)
```

In [63]:
estado = ""
rep = 0

while estado != "OK" and rep < 20:
    respuesta = requests.get(url = url_consulta, headers = headers)
    estado = respuesta.reason
    rep += 1

print(respuesta)
print(respuesta.reason)

<Response [200]>
OK


El response 200 quiere decir que la consulta se hizo satisfactoriamente. Si este no es el valor que te aparece considera las siguientes opciones:

* Si el valor es 202, entonces deberás activar en tu navegador predefinido **JavaScript** (consulta en internet cómo hacerlo). Una vez hecho vuelve a ejecutar el código hasta que aparezca 200.

* Si el valor es otro, prueba modificar el elemento User-Agent de `headers` por uno de los siguientes:

```py
"Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36"
"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"
"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15"
"Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15"
```

* Si ninguno de estos casos funciona, consulta en internet cómo extraer tu User-Agent propio en las herramientas de tu navegador.

Estructura el código *html* que está en el elemento *text* de la variable respuesta usando la función `BeautifulSoup` de la librería `bs4`.

```py
soup = BeautifulSoup(
    # Aquí tu código #, 
    "lxml"
) 
```

In [64]:
soup = BeautifulSoup(respuesta.text, "lxml")

Ya tenemos guardada y estructura la consulta a la url de búsqueda, por lo tanto es posible iniciar con el proceso de extracción de información.

## Extracción de información

En primera instancia conviene descubrir dónde está almacenada la información de cada hotel dentro de la página web. Para esto conviene explorar el código *html* línea por línea (lo cual es un trabajo un tanto tedioso, pero necesario). Puede ayudarte mucho utilizar la opción *Inspeccionar* de tu navegador. 

En general, debes entender que el código *html* de cualquier sitio web se compone de **bloques** anidados que tienen la siguiente forma básica:

```html
<nombre_bloque atributos> texto_u_otros_bloques </nombre_bloque>
```
Y lo que necesitamos ahora es conocer es el **nombre del bloque** y el **atributo referencial** donde se encuentra esta parte del sitio web:

![](card_booking.png)

A partir de esta investigación podemos encontrar este código *html*: 

![](html1.png)

De aquí se obtiene que esta parte del sitio web corresponde al bloque *div* y el atributo *data-testid* con el valor de "property-card". Utiliza esto y el método `find_all` para extraer la información de los hoteles y guardarla en la variable `info_hoteles`. Posteriormente cuenta cuántos hoteles hay en la página con la función `len`.

```py
info_hoteles = soup.find_all(
    name = # Aquí tu código #, 
    attrs = {# Aquí tu código #}
)
len(info_hoteles)
```

In [65]:
info_hoteles = soup.find_all(
    name = "div", 
    attrs = {"data-testid":"property-card"}
)
len(info_hoteles)

25

Extrae y guarda los nombres de los hoteles identificando en qué bloque y atributo *data-testid* se encuentra este dato. Para esto, y considerando que los mismos están anidados en las "property cards*, te recomiendo utilizar un bucle que recorra `info_hoteles` y el método `find`. A continuación el código *html* correspondiente para que extraigas lo que necesitas:

![](html2.png)

**Pistas:**

* El método `find` tiene los mismos atributos que `find_all`, esto es name y attrs.
* Solamente conviene guardar el elemento *text* de la extracción realizada.
* Te dejo además esta plantilla:

```py
nombres = []
for caso in info_hoteles:
    n = # Aquí tu código #
    n = n.text
    nombres.append(n)

print(nombres)
```

In [66]:
nombres = []
for caso in info_hoteles:
    n = caso.find(
        name = "div", 
        attrs = {"data-testid": "title"}
    )
    n = n.text
    nombres.append(n)

print(nombres)

['GO Quito Hotel', 'Swissotel Quito', 'Le Parc Hotel, Beyond Stars', 'JW Marriott Quito', 'Mercure Alameda Quito', 'Hilton Colon Quito Hotel', 'Illa Experience Hotel', 'Hotel Boutique Casa San Marcos', 'NH Collection Quito Royal', 'Mama Cuchara by Art Hotels', 'Plaza Grande Hotel', 'Sheraton Quito', 'Luxury Residence Suites', 'Dann Carlton Quito', 'Hotel Casa Gangotena', 'Hotel Patio Andaluz', 'Hotel Tourblanche', 'Wyndham Quito Airport', 'Holiday Inn - Quito Airport by IHG', 'Eb Hotel By Eurobuilding Quito Airport', 'San Jose de Puembo Quito Airport, an Ascend Collection Hotel', 'Hotel Rincon de Puembo; BW Signature Collection', 'Hacienda Jimenita Wildlife Reserve', 'La Palma Polo Hotel - Airport Area', 'Hacienda Las Cuevas Terra Lodge']


Extrae y guarda la localización específica de los hoteles identificando en qué bloque y atributo *data-testid* se encuentra este dato. Aquí el código *html* correspondiente:

![](html3.png)

In [67]:
localizacion = []
for caso in info_hoteles:
    l = caso.find(
        name = "span", 
        attrs = {"data-testid": "address"}
    )
    l = l.text
    localizacion.append(l)

print(localizacion)

['Bellavista, Quito (Bellavista)', 'La Floresta, Quito', 'La Carolina, Quito (La Carolina)', 'Quito', 'La Mariscal, Quito', 'La Mariscal, Quito', 'Centro histórico, Quito', 'Centro histórico, Quito', 'La Floresta, Quito', 'Centro histórico, Quito', 'Centro histórico, Quito', 'La Carolina, Quito (La Carolina)', 'La Carolina, Quito (La Carolina)', 'La Carolina, Quito (La Carolina)', 'Centro histórico, Quito', 'Centro histórico, Quito', 'Quito', 'Tababela', 'Tababela', 'Tababela', 'Puembo', 'Puembo', 'Puembo', 'Puembo', 'Pifo']


Extrae y guarda el score de los hoteles identificando en qué bloque y atributo se encuentra este dato (ten cuidado porque no todos los hoteles tienen score). Para hacer esto, sigue los pasos descritos: 

1. Primero identifica y extrae de este código *html*, el cuadro donde se encuentran los scores y reviews.

![](html4.png)

2. Luego de aquí extre el score a través del atributo *class*.

![](html5.png)

3. Finalmente, aplica esta plantilla:

```py
scores = []
for caso in info_hoteles:
    rating = # Aquí tu código del Paso 1 #
    if rating != None:
        s = # Aquí tu código del Paso 2 #
        s = s.text
        scores.append(s)
    else:
        scores.append("")

print(scores)
```

In [68]:
scores = []
for caso in info_hoteles:
    rating = caso.find(name = "div", attrs = {"data-testid": "review-score"})
    if rating != None:
        s = rating.find(name = "div", attrs = {"class":"f63b14ab7a dff2e52086"})
        s = s.text
        scores.append(s)
    else:
        scores.append("")

print(scores)

['9,2', '9,0', '8,9', '9,2', '8,8', '8,1', '9,1', '9,1', '9,0', '9,0', '9,1', '8,7', '9,5', '8,9', '9,5', '8,8', '6,7', '9,1', '9,1', '9,6', '9,0', '8,8', '9,1', '8,9', '9,3']


Extrae y guarda la cantidad de reviews de los hoteles identificando en qué bloque y atributo *class* se encuentra este dato. Considera que así como los scores, estos reviews igualmente están en un bloque anidado. A continuación el código *html* correspondiente:

![](html6.png)

In [69]:
reviews = []
for caso in info_hoteles:
    rating = caso.find(name = "div", attrs = {"data-testid": "review-score"})
    if rating != None:
        r = rating.find(name = "div", attrs={"class":"fff1944c52 fb14de7f14 eaa8455879"})
        r = r.text
        reviews.append(r)
    else:
        reviews.append("")

print(reviews)

['469 comentarios', '424 comentarios', '481 comentarios', '261 comentarios', '483 comentarios', '299 comentarios', '45 comentarios', '68 comentarios', '537 comentarios', '31 comentarios', '105 comentarios', '259 comentarios', '110 comentarios', '759 comentarios', '61 comentarios', '143 comentarios', '3 comentarios', '3.234 comentarios', '1.103 comentarios', '464 comentarios', '633 comentarios', '212 comentarios', '70 comentarios', '195 comentarios', '57 comentarios']


Vamos ahora a otra parte del *property card*. Extrae y guarda el tipo de habitación de los hoteles, identificando en qué bloque y atributo *class* se encuentra este dato dentro del código *html* a continuación:

![](html7.png)

Toma en cuenta que no todos los casos tienen esta información disponible para extracción.

In [70]:
habitaciones = []
for caso in info_hoteles:
    h = caso.find(name = "h4", attrs = {"class": "fff1944c52 f254df5361"})
    h = h.text if h != None else ""
    habitaciones.append(h)

print(habitaciones)

['Habitación Doble Estándar con 2 camas grandes', 'Habitación Doble Premier - 1 o 2 camas', 'Suite Junior', 'Habitación Deluxe - Cama extragrande', 'Superior King Room with One King Bed', 'Habitación Doble Deluxe - 2 camas dobles', 'Habitación Doble Deluxe - 1 o 2 camas', 'Habitación con cama extragrande y vistas a la montaña', 'Habitación Doble Superior - 1 o 2 camas', 'Habitación Doble Deluxe', 'Suite Doble Estándar', 'Habitación Doble - 2 camas dobles', 'Apartamento', 'Habitación Doble Superior - 2 camas', 'Habitación Doble Deluxe', 'Habitación con cama extragrande.', 'Habitación Doble - 2 camas', 'Habitación Deluxe - Cama grande - No Fumadores', 'Habitación Estándar con cama extragrande - Adaptada para personas de movilidad reducida', 'Habitación con cama extragrande.', 'Habitación Doble Estándar con vistas al jardín - 2 camas dobles', 'Habitación con cama extragrande y cama individual', 'Habitación Doble Deluxe - 2 camas', 'Habitación con cama extragrande.', 'Habitación Triple con

Extrae y guarda el precio sin descuento de los hoteles, identificando en qué bloque y atributo *class* se encuentra este dato dentro del código *html* a continuación:

![](html8.png)

Toma en cuenta que no todos los casos tienen esta información disponible para extracción.

In [71]:
precio_sd = []
for caso in info_hoteles:
    p = caso.find(name = "span", attrs = {"class": "fff1944c52 d68334ea31 ab607752a2"})
    p = p.text if p != None else ""
    precio_sd.append(p)

print(precio_sd)

['US$270', '', 'US$112', '', '', '', 'US$470', 'US$167', '', '', 'US$400', '', '', 'US$134', '', '', '', '', '', '', '', '', '', '', 'US$142']


Extrae y guarda el precio con descuento de los hoteles identificando en qué bloque y atributo *data-testid* se encuentra este dato. Este es el código *html* a considerar:

![](html9.png)

In [72]:
precio_cd = []
for caso in info_hoteles:
    p = caso.find(name = "span", attrs = {"data-testid": "price-and-discounted-price"})
    p = p.text
    precio_cd.append(p)

print(precio_cd)

['US$189', 'US$165', 'US$106', 'US$161', 'US$95', 'US$145', 'US$413', 'US$142', 'US$98', 'US$261', 'US$300', 'US$184', 'US$65', 'US$115', 'US$568', 'US$171', 'US$55', 'US$171', 'US$114', 'US$140', 'US$115', 'US$127', 'US$155', 'US$200', 'US$135']


Extrae y guarda el valor de impuestos y servicios de los hoteles identificando en qué bloque y atributo *data-testid* se encuentra este dato. Este es el código *html* a considerar:

![](html10.png)

In [73]:
taxes = []
for caso in info_hoteles:
    t = caso.find(name = "div", attrs = {"data-testid": "taxes-and-charges"})
    t = t.text
    taxes.append(t)

print(taxes)

['+ US$50 de impuestos y cargos', '+ US$44 de impuestos y cargos', '+ US$33 de impuestos y cargos', '+ US$43 de impuestos y cargos', '+ US$26 de impuestos y cargos', '+ US$39 de impuestos y cargos', '+ US$106 de impuestos y cargos', '+ US$37 de impuestos y cargos', '+ US$37 de impuestos y cargos', '+ US$29 de impuestos y cargos', '+ US$78 de impuestos y cargos', '+ US$49 de impuestos y cargos', '+ US$22 de impuestos y cargos', '+ US$34 de impuestos y cargos', '+ US$59 de impuestos y cargos', '+ US$19 de impuestos y cargos', '+ US$14 de impuestos y cargos', '+ US$46 de impuestos y cargos', '+ US$30 de impuestos y cargos', '+ US$38 de impuestos y cargos', '+ US$38 de impuestos y cargos', '+ US$34 de impuestos y cargos', '+ US$42 de impuestos y cargos', '+ US$52 de impuestos y cargos', '+ US$36 de impuestos y cargos']


Extrae y guarda como una variable buleana la información respecto a si el hotel incluye o no desayunos, identificando en qué bloque y atributo *class* se encuentra este dato. Este es el código *html* a considerar:

![](html11.png)

In [74]:
incl_desayuno = []
for caso in info_hoteles:
    d = caso.find(name = "span", attrs = {"class": "d651523034"})
    d = False if d != None else True
    incl_desayuno.append(d)

print(incl_desayuno)

[True, True, True, True, True, True, False, False, True, False, False, True, True, False, False, False, False, True, True, True, False, False, False, False, False]


## Consolidación de extracción

Una vez extraída toda la información relevante, conviene integrar todos los datos en un dataframe. Haslo utlizando **pandas**.

In [75]:
df_booking = pd.DataFrame(dict(
    nombre_hotel = nombres,
    localizacion = localizacion,
    score = scores,
    reviews = reviews,
    tipo_habitacion = habitaciones,
    incluye_desayuno = incl_desayuno,
    precio_sin_dscto = precio_sd,
    precio_con_dscto = precio_cd,
    impuestos = taxes
))

df_booking.sample(10)

,nombre_hotel,localizacion,score,reviews,tipo_habitacion,incluye_desayuno,precio_sin_dscto,precio_con_dscto,impuestos
6,Illa Experience Hotel,"Centro histórico, Quito","9,1",45 comentarios,Habitación Doble Deluxe - 1 o 2 camas,False,US$470,US$413,+ US$106 de impuestos y cargos
5,Hilton Colon Quito Hotel,"La Mariscal, Quito","8,1",299 comentarios,Habitación Doble Deluxe - 2 camas dobles,True,,US$145,+ US$39 de impuestos y cargos
12,Luxury Residence Suites,"La Carolina, Quito (La Carolina)","9,5",110 comentarios,Apartamento,True,,US$65,+ US$22 de impuestos y cargos
10,Plaza Grande Hotel,"Centro histórico, Quito","9,1",105 comentarios,Suite Doble Estándar,False,US$400,US$300,+ US$78 de impuestos y cargos
11,Sheraton Quito,"La Carolina, Quito (La Carolina)","8,7",259 comentarios,Habitación Doble - 2 camas dobles,True,,US$184,+ US$49 de impuestos y cargos
21,Hotel Rincon de Puembo; BW Signature Collection,Puembo,"8,8",212 comentarios,Habitación con cama extragrande y cama individual,False,,US$127,+ US$34 de impuestos y cargos
17,Wyndham Quito Airport,Tababela,"9,1",3.234 comentarios,Habitación Deluxe - Cama grande - No Fumadores,True,,US$171,+ US$46 de impuestos y cargos
14,Hotel Casa Gangotena,"Centro histórico, Quito","9,5",61 comentarios,Habitación Doble Deluxe,False,,US$568,+ US$59 de impuestos y cargos
1,Swissotel Quito,"La Floresta, Quito","9,0",424 comentarios,Habitación Doble Premier - 1 o 2 camas,True,,US$165,+ US$44 de impuestos y cargos
20,"San Jose de Puembo Quito Airport, an Ascend Co...",Puembo,"9,0",633 comentarios,Habitación Doble Estándar con vistas al jardín...,False,,US$115,+ US$38 de impuestos y cargos


Vamos ahora a procesar un poco este dataframe para que muestre la información de mejor manera. Por tanto, da tratamiento a los nombres de los hoteles y al tipo de habitación de forma que estén en mayúscula, sin espacios innecesarios y sin caracteres de pausa.

In [76]:
# Ajustar columna nombre_hotel
df_booking["nombre_hotel"] = (
    df_booking["nombre_hotel"]
    .apply(lambda x: re.sub("[-_;:,\\.]", "", x).upper().strip())
    .apply(lambda x: re.sub("[ ]+"," ",x))
)
df_booking["nombre_hotel"].unique()

array(['GO QUITO HOTEL', 'SWISSOTEL QUITO', 'LE PARC HOTEL BEYOND STARS',
       'JW MARRIOTT QUITO', 'MERCURE ALAMEDA QUITO',
       'HILTON COLON QUITO HOTEL', 'ILLA EXPERIENCE HOTEL',
       'HOTEL BOUTIQUE CASA SAN MARCOS', 'NH COLLECTION QUITO ROYAL',
       'MAMA CUCHARA BY ART HOTELS', 'PLAZA GRANDE HOTEL',
       'SHERATON QUITO', 'LUXURY RESIDENCE SUITES', 'DANN CARLTON QUITO',
       'HOTEL CASA GANGOTENA', 'HOTEL PATIO ANDALUZ', 'HOTEL TOURBLANCHE',
       'WYNDHAM QUITO AIRPORT', 'HOLIDAY INN QUITO AIRPORT BY IHG',
       'EB HOTEL BY EUROBUILDING QUITO AIRPORT',
       'SAN JOSE DE PUEMBO QUITO AIRPORT AN ASCEND COLLECTION HOTEL',
       'HOTEL RINCON DE PUEMBO BW SIGNATURE COLLECTION',
       'HACIENDA JIMENITA WILDLIFE RESERVE',
       'LA PALMA POLO HOTEL AIRPORT AREA',
       'HACIENDA LAS CUEVAS TERRA LODGE'], dtype=object)

In [77]:
# Ajustar columna tipo_habitacion
df_booking["tipo_habitacion"] = (
    df_booking["tipo_habitacion"]
    .apply(lambda x: re.sub("[-_;:,\\.]", "", x).upper().strip())
    .apply(lambda x: re.sub("[ ]+"," ",x))
)
df_booking["tipo_habitacion"].unique()

array(['HABITACIÓN DOBLE ESTÁNDAR CON 2 CAMAS GRANDES',
       'HABITACIÓN DOBLE PREMIER 1 O 2 CAMAS', 'SUITE JUNIOR',
       'HABITACIÓN DELUXE CAMA EXTRAGRANDE',
       'SUPERIOR KING ROOM WITH ONE KING BED',
       'HABITACIÓN DOBLE DELUXE 2 CAMAS DOBLES',
       'HABITACIÓN DOBLE DELUXE 1 O 2 CAMAS',
       'HABITACIÓN CON CAMA EXTRAGRANDE Y VISTAS A LA MONTAÑA',
       'HABITACIÓN DOBLE SUPERIOR 1 O 2 CAMAS', 'HABITACIÓN DOBLE DELUXE',
       'SUITE DOBLE ESTÁNDAR', 'HABITACIÓN DOBLE 2 CAMAS DOBLES',
       'APARTAMENTO', 'HABITACIÓN DOBLE SUPERIOR 2 CAMAS',
       'HABITACIÓN CON CAMA EXTRAGRANDE', 'HABITACIÓN DOBLE 2 CAMAS',
       'HABITACIÓN DELUXE CAMA GRANDE NO FUMADORES',
       'HABITACIÓN ESTÁNDAR CON CAMA EXTRAGRANDE ADAPTADA PARA PERSONAS DE MOVILIDAD REDUCIDA',
       'HABITACIÓN DOBLE ESTÁNDAR CON VISTAS AL JARDÍN 2 CAMAS DOBLES',
       'HABITACIÓN CON CAMA EXTRAGRANDE Y CAMA INDIVIDUAL',
       'HABITACIÓN DOBLE DELUXE 2 CAMAS',
       'HABITACIÓN TRIPLE CON VISTAS 

Da tratamiento a las localizaciones de forma que que todos estén en mayúscula, sin espacios innecesarios y borrando todo lo que viene luego de las comas (",").

In [78]:
df_booking["localizacion"] = (
    df_booking["localizacion"]
    .apply(lambda x: re.sub(", [\\w \\(\\)]*", "", x).upper().strip())
)
df_booking["localizacion"].unique()

array(['BELLAVISTA', 'LA FLORESTA', 'LA CAROLINA', 'QUITO', 'LA MARISCAL',
       'CENTRO HISTÓRICO', 'TABABELA', 'PUEMBO', 'PIFO'], dtype=object)

Da tratamiento al score y a los reviews a fin que sean columnas numéricas.

In [79]:
# Ajustar columna score
df_booking["score"] = (
    df_booking["score"]
    .apply(lambda x: re.sub(",",".",x))
    .apply(lambda x: np.nan if x == "" else x)
    .astype(float)
)
df_booking["score"].describe().round(2)

count    25.00
mean      8.94
std       0.55
min       6.70
25%       8.90
50%       9.00
75%       9.10
max       9.60
Name: score, dtype: float64

In [80]:
# Ajustar columna reviews
df_booking["reviews"] = (
    df_booking["reviews"]
    .apply(lambda x: re.sub("\\.","",re.sub(" [\\w]*","",x)))
    .apply(lambda x: np.nan if x == "" else x)
    .astype(int)
)
df_booking["reviews"].describe().round(2)

count      25.00
mean      420.24
std       644.72
min         3.00
25%        70.00
50%       259.00
75%       481.00
max      3234.00
Name: reviews, dtype: float64

Da tratamiento a los precios con y sin descuento, y a los impuestos de forma que todas sean variables numéricas.

In [81]:
# Ajustar columna precios_sin_dscto
df_booking["precio_sin_dscto"] = (
    df_booking["precio_sin_dscto"]
    .apply(lambda x: re.sub("[A-z\\$]*","",x))
    .apply(lambda x: np.nan if x == "" else x)
    .astype(float)
)
df_booking["precio_sin_dscto"].describe().round(2)

count      7.00
mean     242.14
std      142.55
min      112.00
25%      138.00
50%      167.00
75%      335.00
max      470.00
Name: precio_sin_dscto, dtype: float64

In [82]:
# Ajustar columna precios_con_dscto
df_booking["precio_con_dscto"] = (
    df_booking["precio_con_dscto"]
    .apply(lambda x: re.sub("[A-z\\$]*","",x))
    .apply(lambda x: np.nan if x == "" else x)
    .astype(float)
)
df_booking["precio_con_dscto"].describe().round(2)

count     25.00
mean     175.60
std      111.31
min       55.00
25%      115.00
50%      145.00
75%      184.00
max      568.00
Name: precio_con_dscto, dtype: float64

In [83]:
# Ajustar columna impuestos
df_booking["impuestos"] = (
    df_booking["impuestos"]
    .apply(lambda x: re.sub("\\+ [A-z\\$]*","",x))
    .apply(lambda x: re.sub(" [\\w]*","",x))
    .apply(lambda x: np.nan if x == "" else x)
    .astype(float)
)
df_booking["impuestos"].describe().round(2)

count     25.00
mean      41.40
std       18.85
min       14.00
25%       33.00
50%       38.00
75%       46.00
max      106.00
Name: impuestos, dtype: float64

Inputa los valores perdidos de reviews con el valor de 0.

In [84]:
df_booking["reviews"] = df_booking["reviews"].fillna(0)
df_booking["reviews"].describe().round(2)

count      25.00
mean      420.24
std       644.72
min         3.00
25%        70.00
50%       259.00
75%       481.00
max      3234.00
Name: reviews, dtype: float64

Imputa los valores perdidos de precio_sin_dscto con los valores de la columna precio_con_dscto.

In [85]:
df_booking["precio_sin_dscto"] = df_booking["precio_sin_dscto"].fillna(df_booking["precio_con_dscto"])
df_booking["precio_sin_dscto"].describe().round(2)

count     25.00
mean     187.40
std      123.07
min       55.00
25%      115.00
50%      155.00
75%      184.00
max      568.00
Name: precio_sin_dscto, dtype: float64

Calcula el porcentaje de descuento aplicado en una nueva columna. El descuento lo puedes calcular con la fórmula:

$$ descuento = 1-\frac{precio\;con\;descuento}{precio\;sin\;descuento} $$

In [86]:
df_booking["pct_dscto"] = 1 - df_booking["precio_con_dscto"] / df_booking["precio_sin_dscto"]
df_booking["pct_dscto"].describe().round(2)

count    25.00
mean      0.04
std       0.08
min       0.00
25%       0.00
50%       0.00
75%       0.05
max       0.30
Name: pct_dscto, dtype: float64

¡Listo! hemos extraído la información y la hemos estructurado en un dataframe. Imprime una muestra de estos datos para concluir.

In [87]:
df_booking.sample(10)

,nombre_hotel,localizacion,score,reviews,tipo_habitacion,incluye_desayuno,precio_sin_dscto,precio_con_dscto,impuestos,pct_dscto
23,LA PALMA POLO HOTEL AIRPORT AREA,PUEMBO,8.9,195,HABITACIÓN CON CAMA EXTRAGRANDE,False,200.0,200.0,52.0,0.000000
2,LE PARC HOTEL BEYOND STARS,LA CAROLINA,8.9,481,SUITE JUNIOR,True,112.0,106.0,33.0,0.053571
20,SAN JOSE DE PUEMBO QUITO AIRPORT AN ASCEND COL...,PUEMBO,9.0,633,HABITACIÓN DOBLE ESTÁNDAR CON VISTAS AL JARDÍN...,False,115.0,115.0,38.0,0.000000
3,JW MARRIOTT QUITO,QUITO,9.2,261,HABITACIÓN DELUXE CAMA EXTRAGRANDE,True,161.0,161.0,43.0,0.000000
9,MAMA CUCHARA BY ART HOTELS,CENTRO HISTÓRICO,9.0,31,HABITACIÓN DOBLE DELUXE,False,261.0,261.0,29.0,0.000000
1,SWISSOTEL QUITO,LA FLORESTA,9.0,424,HABITACIÓN DOBLE PREMIER 1 O 2 CAMAS,True,165.0,165.0,44.0,0.000000
8,NH COLLECTION QUITO ROYAL,LA FLORESTA,9.0,537,HABITACIÓN DOBLE SUPERIOR 1 O 2 CAMAS,True,98.0,98.0,37.0,0.000000
11,SHERATON QUITO,LA CAROLINA,8.7,259,HABITACIÓN DOBLE 2 CAMAS DOBLES,True,184.0,184.0,49.0,0.000000
10,PLAZA GRANDE HOTEL,CENTRO HISTÓRICO,9.1,105,SUITE DOBLE ESTÁNDAR,False,400.0,300.0,78.0,0.250000
4,MERCURE ALAMEDA QUITO,LA MARISCAL,8.8,483,SUPERIOR KING ROOM WITH ONE KING BED,True,95.0,95.0,26.0,0.000000


Si gustas puedes guardar tu resultado localmente con el método `to_csv` de **pandas**.